In [6]:
import pandas as pd
import numpy as np
import subprocess
import os


In [7]:
phenotypes = pd.read_csv('../data/std_GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz', compression='gzip', sep='\t')
filtered_phenotypes = phenotypes[~phenotypes['Chr'].isin(['X', 'Y', 'M'])]
to_calc = filtered_phenotypes[['TargetID', 'Chr']]
to_calc = to_calc[to_calc['Chr'].isin(['19'])].loc[6532:]
to_calc

,TargetID,Chr
6532,ENSG00000105492.10,19
6553,ENSG00000105499.8,19
6605,ENSG00000186230.5,19
6606,ENSG00000126934.6,19
6628,ENSG00000186806.3,19
...,...,...
23604,ENSG00000132017.5,19
23627,ENSG00000167676.3,19
23638,ENSG00000051128.12,19
23643,ENSG00000142396.4,19


In [8]:
to_calc[to_calc['TargetID'] == 'ENSG00000105492.10']
to_calc.loc[6532]

TargetID    ENSG00000105492.10
Chr                         19
Name: 6532, dtype: object

In [9]:
phenotypes

,TargetID,Gene_Symbol,Chr,Coord,HG00278,HG00108,HG00320,HG00329,HG00122,NA12843,...,HG00236,NA20515,HG00350,HG00277,HG00269,HG00178,NA20589,NA12275,HG00381,HG00143
0,ENSG00000152931.6,ENSG00000152931.6,5,59783540,-1.181455,0.078751,-0.552245,3.823340,0.050062,1.316995,...,-1.335851,-0.066335,-0.608687,-1.375733,-0.391376,-0.153586,0.697910,-0.117038,-0.765984,-0.492916
1,ENSG00000183696.9,ENSG00000183696.9,7,48128225,1.345727,-0.772473,0.836906,-1.093879,-0.965727,0.407445,...,0.403380,0.272379,0.282127,0.651690,0.147498,0.327500,1.669583,-0.229001,0.085963,-1.123421
2,ENSG00000139269.2,ENSG00000139269.2,12,57846106,-1.625422,-0.865010,0.998292,-0.285025,0.084908,1.510262,...,0.178435,-0.314741,1.089416,2.160462,0.432407,3.047156,-0.476618,0.533370,1.396124,-0.310189
3,ENSG00000169129.8,ENSG00000169129.8,10,116164515,-0.022923,-0.052123,-0.001954,-0.245014,0.010754,-0.138717,...,-0.085691,-0.445712,0.114251,0.530020,-0.286033,-0.025508,-0.260210,-0.211904,-0.182246,0.100830
4,ENSG00000134602.11,ENSG00000134602.11,X,131157293,-0.039043,0.035362,0.014978,-2.441110,0.896749,0.256298,...,1.186627,-1.412918,0.097325,-1.136695,-1.296509,-1.410677,-0.127625,-0.078804,0.146377,0.461997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23717,ENSG00000235472.1,ENSG00000235472.1,13,29172970,0.736629,-0.925722,0.239454,-3.805844,-1.177157,-1.455647,...,0.529020,0.274876,0.411604,0.514605,1.025789,1.623428,0.340317,-0.954270,0.298504,-1.082628
23718,ENSG00000114423.14,ENSG00000114423.14,3,105588396,-0.288377,-0.658505,-0.179214,-1.303185,1.199063,-0.804140,...,-0.543939,-1.068589,-0.026881,-0.334185,0.193377,-0.430277,-0.715513,-1.051626,-0.953878,-0.113589
23719,ENSG00000243312.2,ENSG00000243312.2,4,87791344,0.662080,0.413423,-0.210085,-1.587171,-0.590873,0.035996,...,-0.113964,-1.336663,-0.926202,-0.121694,0.435916,0.975159,0.776673,1.169867,2.312755,1.052995
23720,ENSG00000257337.1,ENSG00000257337.1,12,53448222,1.073795,1.272744,0.510107,-2.627666,-1.011339,-0.889063,...,-0.736361,-0.583468,-1.357720,0.852120,-1.604863,1.580906,0.105125,0.605312,-1.851851,-0.247751


In [ ]:
import os
import pandas as pd
import subprocess

# Define constants for paths and parameters
RSCRIPT_PATH = "Rscript"
FUSION_SCRIPT = "fusion_twas-master/FUSION.compute_weights.R"
BFILE_EQTL_TEMPLATE = "../data/LDREF/1000G.EUR.{chr}"  # Raw genotype data
BFILE_TEMPLATE = "../data/LDREF_cis_eqtls/1000G.EUR.{chr}"  # Placeholder for chromosome
TMP_DIR = "temp_files/"
OUTPUT_DIR = "compute_weights_out/chr_{chr}"
MODELS = "enet,lasso"
PLINK_PATH = "plink.exe"
GCTA_PATH = "fusion_twas-master/gcta_nr_robust.exe"
PHENO_DIR = "../data/gene_expressions/chr_{chr}/"
CIS_WINDOW = 500_000  # 500kb cis-window
MAF_THRESHOLD = 0.01  # Minor allele frequency filter
HWE_THRESHOLD = 1e-6  # Hardy-Weinberg filter
LD_PRUNE_WINDOW = 50  # LD pruning window size
LD_PRUNE_STEP = 5  # LD pruning step size
LD_PRUNE_R2 = 0.2  # LD pruning R^2 threshold
VERBOSE = 2

# Ensure necessary directories exist
os.makedirs("../data/LDREF_cis_eqtls", exist_ok=True)
os.makedirs(TMP_DIR, exist_ok=True)

def filter_cis_eqtls(gene, chromosome):
    """Filters cis-eQTL SNPs and applies LD pruning."""
    CIS_BFILE_TEMPLATE = "../data/LDREF_cis_eqtls/1000G.EUR.{chr}"
    
    # Load gene position information
    gene_info = phenotypes.loc[phenotypes["TargetID"] == gene]

    if gene_info.empty:
        print(f"Warning: Gene {gene} not found in gene position file.")
        return None

    gene_mid = gene_info["Coord"].values[0]

    # Define cis-region boundaries
    cis_start = max(0, gene_mid - CIS_WINDOW)
    cis_end = gene_mid + CIS_WINDOW

    # Define output PLINK file name
    raw_cis_bfile = CIS_BFILE_TEMPLATE.format(chr=chromosome) + "_raw"
    filtered_cis_bfile = CIS_BFILE_TEMPLATE.format(chr=chromosome) + "_filtered"
    pruned_cis_bfile = CIS_BFILE_TEMPLATE.format(chr=chromosome)

    # Step 1: Extract cis-eQTLs
    command_extract = [
        PLINK_PATH,
        "--bfile", BFILE_EQTL_TEMPLATE.format(chr=chromosome),
        "--chr", str(chromosome),
        "--from-bp", str(cis_start),
        "--to-bp", str(cis_end),
        "--make-bed",
        "--out", raw_cis_bfile
    ]

    # Step 2: Apply MAF and HWE filters
    command_qc = [
        PLINK_PATH,
        "--bfile", raw_cis_bfile,
        "--maf", str(MAF_THRESHOLD),
        "--hwe", str(HWE_THRESHOLD),
        "--make-bed",
        "--out", filtered_cis_bfile
    ]

    # Step 3: Prune SNPs to reduce LD
    command_prune = [
        PLINK_PATH,
        "--bfile", filtered_cis_bfile,
        "--indep-pairwise", str(LD_PRUNE_WINDOW), str(LD_PRUNE_STEP), str(LD_PRUNE_R2),
        "--out", filtered_cis_bfile  # Generates a .prune.in file
    ]
    
    command_apply_prune = [
        PLINK_PATH,
        "--bfile", filtered_cis_bfile,
        "--extract", filtered_cis_bfile + ".prune.in",
        "--make-bed",
        "--out", pruned_cis_bfile
    ]

    try:
        subprocess.run(command_extract, check=True)  # Extract cis-eQTLs
        subprocess.run(command_qc, check=True)  # Apply MAF & HWE filters
        subprocess.run(command_prune, check=True)  # Generate pruning list
        subprocess.run(command_apply_prune, check=True)  # Apply pruning
        return pruned_cis_bfile  # Return the path to the cleaned PLINK files
    except subprocess.CalledProcessError as e:
        print(f"Error filtering cis-eQTL PLINK files for gene {gene}, chromosome {chromosome}: {e}")
        return None

def compute_weights(gene, chromosome):
    """Runs the FUSION TWAS pipeline for a specific gene and chromosome."""
    # Apply cis-eQTL filtering and pruning
    cis_bfile = filter_cis_eqtls(gene, chromosome)
    if cis_bfile is None:
        return  # Skip computation if filtering fails

    output_dir = OUTPUT_DIR.format(chr=chromosome)
    os.makedirs(output_dir, exist_ok=True)

    tmp_file_prefix = os.path.join(TMP_DIR, f"test_chr{chromosome}_{gene}")
    output_path = os.path.join(output_dir, f"{gene}_chr{chromosome}")
    pheno_file = os.path.join(PHENO_DIR.format(chr=chromosome), f"{gene}.txt")

    # Construct the command for FUSION
    command = [
        RSCRIPT_PATH, FUSION_SCRIPT,
        "--bfile", cis_bfile,  # Use cleaned cis-eQTL PLINK files
        "--tmp", tmp_file_prefix,
        "--out", output_path,
        "--models", MODELS,
        "--PATH_plink", PLINK_PATH,
        "--PATH_gcta", GCTA_PATH,
        "--verbose", str(VERBOSE),
        "--pheno", pheno_file
    ]
    
    clear_temp_files_command = ['rm', '-rf', 'temp_files/*']
    clear_cis_eqtls_files_command = ['rm', '-rf', '../data/LDREF_cis_eqtls/*']

    # Print command for debugging
    print(f"Running: {' '.join(command)}")
    
    # Run the command
    try:
        subprocess.run(command, check=True)
        subprocess.run(clear_temp_files_command, check=True)
        subprocess.run(clear_cis_eqtls_files_command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running command for gene {gene}, chromosome {chromosome}: {e}")

# Run for each gene in the dataset
for i in to_calc.index:
    gene_data = to_calc.loc[i]
    compute_weights(gene_data['TargetID'], gene_data['Chr'])


Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_cis_eqtls/1000G.EUR.19 --tmp temp_files/test_chr19_ENSG00000105492.10 --out compute_weights_out/chr_19\ENSG00000105492.10_chr19 --models enet,lasso --PATH_plink plink.exe --PATH_gcta fusion_twas-master/gcta_nr_robust.exe --verbose 2 --pheno ../data/gene_expressions/chr_19/ENSG00000105492.10.txt
Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_cis_eqtls/1000G.EUR.19 --tmp temp_files/test_chr19_ENSG00000105499.8 --out compute_weights_out/chr_19\ENSG00000105499.8_chr19 --models enet,lasso --PATH_plink plink.exe --PATH_gcta fusion_twas-master/gcta_nr_robust.exe --verbose 2 --pheno ../data/gene_expressions/chr_19/ENSG00000105499.8.txt
Running: Rscript fusion_twas-master/FUSION.compute_weights.R --bfile ../data/LDREF_cis_eqtls/1000G.EUR.19 --tmp temp_files/test_chr19_ENSG00000186230.5 --out compute_weights_out/chr_19\ENSG00000186230.5_chr19 --models enet,lasso --PATH_plink pli